Title: DSC350 Project  
Author: Caleb Trimble  
Date: 13 November 2024  
Description: This program ingests and transforms multiple data sources from different data types to show the impact of tornadoes, along with the correlations between them and the damage they cause.  
Codes in this program have been adapted from Python Data Analysis-Second Edition (Fandango A., 2017) and Hands-On Data Analysis with Pandas - Second Edition (Molin S., 2021).

In [11]:
import pandas as pd

df = pd.read_csv('1950-2019_all_tornadoes.csv')
df.head()
print(df.columns)
# Transformation 1 - Creates DataFrame and drops columns that are irrelevant for the intended purpose.
df_coldrop = df.drop(columns=['om', 'stf', 'stn', 'tz', 'ns', 'sn', 'sg', 'mo', 'dy', 'f1','f2','f3','f4','fc'])
# Transformation 2 - Creates a column for datetime by combining 'date' and 'time' columns. Defines format for time, and drops the original columns.
df_coldrop['datetime'] = pd.to_datetime(df_coldrop['date'] + ' ' + df_coldrop['time'], format='%Y-%m-%d %H:%M:%S')
df_coldrop.drop(columns=['date', 'time'], inplace=True)
# Transformation 3 - Renames abbreviated columns for uniformity.
df_coldrop.rename(columns={
    'closs': 'Crop Damage',
    'loss': 'Property Damage',
    'yr' : 'Year',
    'st' : 'state',
    'inj' : 'injuries',
    'fat' : 'fatalities',
    'len' : 'length',
    'wid' : 'width',
    'mag' : 'F-Scale'
}, inplace=True)
# Transformation 4 - Combines lat and lon into a single column as tuples and drops the original columns
df_coldrop.loc[:, 'start_coords'] = df_coldrop.apply(lambda row: (row['slat'], row['slon']), axis=1)
df_coldrop.loc[:, 'end_coords'] = df_coldrop.apply(lambda row: (row['elat'], row['elon']), axis=1)
df_coldrop.drop(columns=['slat', 'slon', 'elat', 'elon'], inplace=True)
# Transformation 5 - Changes the order of columns to read more fluently.
ordered = ['Year', 'state', 'start_coords', 'end_coords', 'datetime', 'length', 'width', 'F-Scale', 'injuries', 'fatalities', 'Crop Damage', 'Property Damage']
df_coldrop = df_coldrop[ordered]
df_coldrop.to_csv('tornado_flat.csv', index=False)
print("Flat data has been saved to tornado_flat.csv")
print(df_coldrop)



Index(['om', 'yr', 'mo', 'dy', 'date', 'time', 'tz', 'st', 'stf', 'stn', 'mag',
       'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'len',
       'wid', 'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4', 'fc'],
      dtype='object')
Flat data has been saved to tornado_flat.csv
       Year state         start_coords           end_coords  \
0      1950    MO      (38.77, -90.22)      (38.83, -90.03)   
1      1950    MO      (38.77, -90.22)      (38.82, -90.12)   
2      1950    IL      (38.82, -90.12)      (38.83, -90.03)   
3      1950    IL        (39.1, -89.3)      (39.12, -89.23)   
4      1950    OH      (40.88, -84.58)           (0.0, 0.0)   
...     ...   ...                  ...                  ...   
66383  2019    MS  (33.1628, -89.4323)  (33.2339, -89.3298)   
66384  2019    MS  (33.2598, -89.2778)  (33.2879, -89.2208)   
66385  2019    MS   (33.472, -89.0315)   (33.4888, -88.991)   
66386  2019    MS  (32.5268, -89.1628)  (32.5581, -89.1215)   
66387  2019    AL  (

In [9]:
import requests
import pandas as pd

# Base URL of the API
url = 'https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/Tornado_Tracks_1950_2017_1/FeatureServer/0/query'

# Parameters to handle pagination and ensure proper query
params = {
    'where': 'yr >= 2009 AND yr <= 2019',  # Filter by year range
    'outFields': '*',
    'outSR': 4326,
    'f': 'json',
    'resultRecordCount': 1000,  # Limit for each request
    'resultOffset': 0  # Starting point for records
}

# List to collect all data
all_data = []

# Loop to paginate through the API response
while True:
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        features = data.get('features', [])
        
        if not features:
            break  # Break the loop if no more results
        
        attributes = [feature['attributes'] for feature in features]
        all_data.extend(attributes)
        
        # Update the result offset for the next page
        params['resultOffset'] += params['resultRecordCount']
    else:
        print(f'Error: {response.status_code}')
        break

# Convert the collected data to a DataFrame
df_api = pd.DataFrame(all_data)
print(df_api.columns)

# Continue only if there are columns to drop
if not df_api.empty:
    # Adjust column names to match what's in the DataFrame
    df_api.columns = map(str.lower, df_api.columns)  # Standardize to lower case for consistency

    # Remove the first row of the HTML dataset
    df_api.drop(index=0, inplace=True)

    # Check which columns exist in the DataFrame
    existing_columns = df_api.columns.tolist()
    columns_to_drop = [
        'objectid', 'om', 'mo', 'dy', 'tz', 'stf', 'month_calc', 'date_calc', 'shape__length'
    ]
    columns_to_drop = [col for col in columns_to_drop if col in existing_columns]
    
    # Drop irrelevant columns
    df_api_coldrop = df_api.drop(columns=columns_to_drop, errors='ignore')
    
    # Ensure 'date' and 'time' columns are present before combining
    if 'date' in df_api_coldrop.columns and 'time' in df_api_coldrop.columns:
        # Transformation 2 - Creates a column for datetime by combining 'date' and 'time' columns
        df_api_coldrop['datetime'] = pd.to_datetime(df_api_coldrop['date'] + ' ' + df_api_coldrop['time'], format='%Y-%m-%d %H:%M:%S')
        df_api_coldrop.drop(columns=['date', 'time'], inplace=True)
    
    # Transformation 3 - Renames abbreviated columns for uniformity
    df_api_coldrop.rename(columns={
        'closs': 'Crop Damage',
        'loss': 'Property Damage',
        'yr': 'Year',
        'st': 'state',
        'inj': 'injuries',
        'fat': 'fatalities',
        'len': 'length',
        'wid': 'width',
        'mag': 'F-Scale'
    }, inplace=True, errors='ignore')
    
    # Transformation 4 - Combines lat and lon into a single column as tuples and drops the original columns
    df_api_coldrop.loc[:, 'start_coords'] = df_api_coldrop.apply(lambda row: (row['slat'], row['slon']), axis=1)
    df_api_coldrop.loc[:, 'end_coords'] = df_api_coldrop.apply(lambda row: (row['elat'], row['elon']), axis=1)
    df_api_coldrop.drop(columns=['slat', 'slon', 'elat', 'elon'], inplace=True, errors='ignore')
    
    # Transformation 5 - Changes the order of columns to read more fluently
    ordered = ['Year', 'state', 'start_coords', 'end_coords', 'datetime', 'length', 'width', 'F-Scale', 'injuries', 'fatalities', 'Crop Damage', 'Property Damage']
    df_api_coldrop = df_api_coldrop[ordered]
    
    # Save the transformed DataFrame to a CSV file
    df_api_coldrop.to_csv('tornado_API.csv', index=False)
    print("API data has been saved to tornado_API.csv")
    
    print(df_api_coldrop.head())
else:
    print("No data retrieved from API.")


Index(['OBJECTID', 'om', 'yr', 'mo', 'dy', 'date', 'time', 'tz', 'st', 'stf',
       'stn', 'mag', 'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat',
       'elon', 'len', 'wid', 'fc', 'Month_Calc', 'Date_Calc', 'Shape__Length'],
      dtype='object')
API data has been saved to tornado_API.csv
   Year state         start_coords           end_coords            datetime  \
1  2009    GA       (32.0, -84.25)      (31.99, -84.22) 2009-10-15 17:00:00   
2  2009    LA  (29.7605, -93.0051)  (29.7605, -93.0051) 2009-10-22 09:32:00   
3  2009    LA  (31.9969, -93.4688)  (32.0037, -93.4668) 2009-10-22 10:20:00   
4  2009    LA      (30.05, -92.75)  (30.1647, -92.7268) 2009-10-22 11:00:00   
5  2009    LA   (30.2132, -92.716)  (30.4479, -92.7097) 2009-10-22 11:22:00   

   length  width  F-Scale  injuries  fatalities  Crop Damage  Property Damage  
1    2.11    100        1         0           0          0.0             0.13  
2    0.10     10        0         0           0          0.0      

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch and Parse HTML
url = 'https://data.usatoday.com/tornado-archive/'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Using table_id, searches for the called on table
    table = soup.find('table', {'id': 'tornadoSummary'})
    
    if table:
        # Extracts Table Data
        rows = table.find_all('tr')
        
        data = []
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols if ele.text.strip()]
            if cols:
                data.append(cols)
        
        # Adjusts columns based on actual data structure
        columns = ['Year', 'Number of Tornadoes', 'Direct Injury', 'Indirect Injury', 'Direct Fatality', 'Indirect Fatality', 'Property Damage', 'Crop Damage']

        # Transformation 1 - Creates DataFrame from table data.
        html_df = pd.DataFrame(data, columns=columns)
        html_df.drop(index=0, inplace=True)
        # Transformation 2 - Merges direct and indirect Columns
        html_df['injuries'] = pd.to_numeric(html_df['Direct Injury'], errors='coerce') + pd.to_numeric(html_df['Indirect Injury'], errors='coerce')
        html_df['fatalities'] = pd.to_numeric(html_df['Direct Fatality'], errors='coerce') + pd.to_numeric(html_df['Indirect Fatality'], errors='coerce')

        # Transformation 3 - drops unnecessary columns.  
        html_df.drop(columns=['Direct Injury', 'Indirect Injury', 'Direct Fatality', 'Indirect Fatality'], inplace=True)

        # Transformation 4 - Cleans and converts Damage Columns
        html_df['Property Damage'] = html_df['Property Damage'].str.replace(',', '').str.replace('$', '') 
        html_df['Crop Damage'] = html_df['Crop Damage'].str.replace(',', '').str.replace('$', '')

        # Transformation 5 - Converts datatypes to Numeric
        html_df['Property Damage'] = pd.to_numeric(html_df['Property Damage'], errors='coerce')
        html_df['Crop Damage'] = pd.to_numeric(html_df['Crop Damage'], errors='coerce')
        # Save the transformed DataFrame to a CSV file
        html_df.to_csv('tornado_html.csv', index=False)
        print("HTML data has been saved to tornado_html.csv")

        print(html_df.head())
    else:
        print("Failed to locate the table with id 'tornadoSummary' on the webpage.")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


HTML data has been saved to html_data_transformed.csv
   Year Number of Tornadoes  Property Damage  Crop Damage  injuries  \
1  2024               1,664        935078800   13326200.0     544.0   
2  2023               1,523       1371376500    7300800.0     955.0   
3  2022               1,384        698683090    5550500.0     318.0   
4  2021               1,545        232623000    2238400.0     881.0   
5  2020               1,251       2504035500   27261500.0     741.0   

   fatalities  
1          45  
2          91  
3          25  
4         107  
5          77  


In [6]:
import sqlite3
import pandas as pd

# Load the transformed flat file CSV into SQLite3
df_flatfile = pd.read_csv('tornado_flat.csv')

# Ensure proper connection handling
with sqlite3.connect('DSC350_Project.db', timeout=10) as conn:
    df_flatfile.to_sql('FlatFile_Table', conn, if_exists='replace', index=False)

# Load the API CSV into SQLite3
df_api = pd.read_csv('tornado_API.csv')

# Ensure proper connection handling
with sqlite3.connect('DSC350_Project.db') as conn:
    df_api.to_sql('API_Table', conn, if_exists='replace', index=False)

# Load the HTML CSV into SQLite3
df_html = pd.read_csv('tornado_html.csv')

# Ensure proper connection handling
with sqlite3.connect('DSC350_Project.db') as conn:
    df_html.to_sql('HTML_Table', conn, if_exists='replace', index=False)

# Query to merge the datasets
query = '''
    SELECT *
    FROM FlatFile_Table AS ff
    JOIN API_Table AS api ON ff.year = api.year
    JOIN HTML_Table AS html ON ff.year = html.Year
'''

# Process the query in smaller chunks
chunks = []
with sqlite3.connect('DSC350_Project.db') as conn:
    for chunk in pd.read_sql_query(query, conn, chunksize=5000):  # Adjust chunk size as needed
        chunks.append(chunk)

merged_df = pd.concat(chunks, ignore_index=True)

# Display the merged dataset
print(merged_df.head())


C:\Users\caleb\AppData\Local\Temp\ipykernel_15192\2895886704.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(chunks, ignore_index=True)


   year state       start_coords         end_coords             datetime  \
0  2009    MS  (31.875, -89.331)  (31.888, -89.327)  2009-01-03 13:13:00   
1  2009    MS  (31.875, -89.331)  (31.888, -89.327)  2009-01-03 13:13:00   
2  2009    MS  (31.875, -89.331)  (31.888, -89.327)  2009-01-03 13:13:00   
3  2009    MS  (31.875, -89.331)  (31.888, -89.327)  2009-01-03 13:13:00   
4  2009    MS  (31.875, -89.331)  (31.888, -89.327)  2009-01-03 13:13:00   

   length  width F-Scale  injuries  fatalities  ...  width F-Scale  injuries  \
0    0.93    175       1         0           0  ...     50       0         0   
1    0.93    175       1         0           0  ...   1000       1         0   
2    0.93    175       1         0           0  ...    200       1         0   
3    0.93    175       1         0           0  ...     30       0         0   
4    0.93    175       1         0           0  ...     50       0         0   

   fatalities  Year  Number of Tornadoes  Property Damage  Cro

Note of datasource change:  
I was unhappy with the data provided on the opendatasoft API, so I chose to change the API to FEMA to ensure data integrity. 